In [ ]:
#Map with risks and solutions

In [7]:
import pandas as pd
import plotly.graph_objs as go
import dash 

In [2]:
import pandas as pd
import plotly.graph_objs as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, ALL

# Read the data from the Excel file
excel_url='https://github.com/ashariq12/CCDR/raw/main/CCDR_adaptation_topic_results.xlsx' 
df = pd.read_excel(excel_url)

# Filter topics based on category
adaptation_topics = df[df['topic_category'] == 'adaptation and resilience solution']['topic'].unique()
cross_cutting_topics = df[df['topic_category'] == 'cross-cutting impacts']['topic'].unique()
climate_risks_topics = df[df['topic_category'] == 'climate risk']['topic'].unique()

# Define symbols for climate risks
symbols = {
    'Floods': 'circle',
    'Drought': 'square',
    'Extreme winds': 'diamond',
    'Urban heat': 'triangle-up',
    'Extreme heat': 'triangle-down'
}

# Define colors for climate risks
colors = {
    'Floods': 'blue',
    'Drought': 'orange',
    'Extreme winds': 'green',
    'Urban heat': 'red',
    'Extreme heat': 'red'
}

# Create Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1("CCDR Climate Risks and A&R Solutions Search Tool", style={'color': 'darkblue', 'fontFamily': 'Calibri, sans-serif'}),
    html.Div([
        html.Label("Select Climate Risks:"),
        *[html.Div([dcc.Checklist(id={'type': 'climate-risk-checkbox', 'index': idx}, options=[{'label': topic, 'value': topic}], value=[])]) for idx, topic in enumerate(climate_risks_topics)]
    ]),
    html.Br(),
    html.Div(id='dropdown-container', children=[
        html.Label("Select Topic Type:"),
        dcc.RadioItems(
            id='category-radio',
            options=[
                {'label': 'Adaptation and Resilience Solutions', 'value': 'adaptation'},
                {'label': 'Cross-Cutting Impacts', 'value': 'cross_cutting'}
            ],
            value='adaptation',
            labelStyle={'display': 'inline-block'}
        ),
        html.Div(id='topic-dropdown-container'),
        # Initialize hidden topic dropdown
        dcc.Dropdown(id='topic-dropdown', style={'display': 'none'})
    ]),
    dcc.Graph(id='world-map')
])

# Define callback to update topic dropdown based on selected category
@app.callback(
    Output('topic-dropdown-container', 'children'),
    [Input('category-radio', 'value')]
)
def update_dropdown(category):
    if category == 'adaptation':
        options = [{'label': topic, 'value': topic} for topic in adaptation_topics]
    elif category == 'cross_cutting':
        options = [{'label': topic, 'value': topic} for topic in cross_cutting_topics]
    else:
        options = []

    return html.Div([
        dcc.Dropdown(
            id='topic-dropdown',
            options=options,
            value=options[0]['value'] if options else None,
            clearable=False
        )
    ])

# Define callback to update the world map
@app.callback(
    Output('world-map', 'figure'),
    [Input('topic-dropdown', 'value')] +
    [Input({'type': 'climate-risk-checkbox', 'index': ALL}, 'value')]
)
def update_map(selected_topic, selected_climate_risks_topics):
    if selected_topic is None:
        return go.Figure(go.Choropleth())

    # Filter the dataframe based on selected topic
    filtered_df = df[df['topic'] == selected_topic]

    # Get the countries where the selected topic appears
    highlighted_countries = filtered_df[filtered_df['count'] > 0]['country'].tolist()

    # Create a choropleth map
    fig = go.Figure(go.Choropleth(
        locations=highlighted_countries,  # Highlighted countries
        z=filtered_df[filtered_df['count'] > 0]['count'],  # Frequency of the selected topic for highlighted countries
        locationmode='country names',  # Use country names for locations
        colorscale='Viridis',
        autocolorscale=True,  # Automatically scale the color based on the z values
        reversescale=True,
        marker_line_color='darkgray',
        marker_line_width=0.5,
        colorbar_title='Frequency'
    ))

    fig.update_layout(
        title=f"CCDRs that contain material on '{selected_topic}'",
        geo=dict(
            showframe=False,
            showcoastlines=True,
            projection_type='equirectangular'
        )
    )

    # Add symbols for selected climate risk topics if any are selected
    for idx, topic in enumerate(climate_risks_topics):
        if idx < len(selected_climate_risks_topics) and selected_climate_risks_topics[idx]:
            # Filter the dataframe for the selected climate risk topic
            climate_risk_filtered_df = df[df['topic'] == topic]
            # Get the countries where the selected climate risk topic appears
            climate_risk_highlighted_countries = climate_risk_filtered_df[climate_risk_filtered_df['count'] > 0]['country'].tolist()
            fig.add_trace(go.Scattergeo(
                locationmode='country names',
                locations=climate_risk_highlighted_countries,
                mode='markers',
                marker=dict(
                    size=6,  # Adjust size here
                    color=colors.get(topic, 'red'),  # Get color for the topic or default to red
                    opacity=0.8,
                    symbol=symbols.get(topic, 'circle')  # Assign symbol for each topic
                ),
                name=topic
            ))

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8050)

